In [1]:
%load_ext autoreload
%autoreload 2

import evals2

import numpy as np
import scipy
import torch
from torch import nn

SEED = 0
np.random.seed(SEED)
torch.manual_seed(SEED)

from matplotlib import pyplot as plt
from pandas import DataFrame
import seaborn as sns
sns.set(font_scale=1.5)
sns.set_style("ticks", {'font.family': 'serif'})
#plt.tight_layout()

SEP = '__'

In [2]:
unigrams = set()
bigrams = {}
with open('rep2_data/reddy.txt') as f:
    next(f) # header
    for line in f:
        line = line.split()
        w1, w2 = line[:2]
        sim = float(line[6])
        w1, _ = w1.split('-')
        w2, _ = w2.split('-')
        bigrams[w1, w2] = sim
        unigrams.add(w1)
        unigrams.add(w2)

In [4]:
bigrams

{('end', 'user'): 4.25,
 ('firing', 'line'): 1.703704,
 ('game', 'plan'): 3.827586,
 ('application', 'form'): 4.8,
 ('snail', 'mail'): 1.310345,
 ('web', 'site'): 3.785714,
 ('flea', 'market'): 1.517241,
 ('grandfather', 'clock'): 2.642857,
 ('diamond', 'wedding'): 1.703704,
 ('guilt', 'trip'): 2.185185,
 ('silver', 'spoon'): 1.518519,
 ('rush', 'hour'): 3.333333,
 ('china', 'clay'): 3.851852,
 ('cocktail', 'dress'): 3.038462,
 ('mailing', 'list'): 4.666667,
 ('spelling', 'bee'): 2.448276,
 ('public', 'service'): 4.4,
 ('head', 'teacher'): 4.0,
 ('video', 'game'): 4.6,
 ('spinning', 'jenny'): 2.275862,
 ('think', 'tank'): 2.038462,
 ('cash', 'cow'): 1.555556,
 ('swan', 'song'): 0.827586,
 ('monkey', 'business'): 0.724138,
 ('snake', 'oil'): 0.75,
 ('smoking', 'gun'): 0.714286,
 ('balance', 'sheet'): 3.862069,
 ('zebra', 'crossing'): 1.25,
 ('cutting', 'edge'): 1.25,
 ('radio', 'station'): 4.466667,
 ('search', 'engine'): 3.32,
 ('shrinking', 'violet'): 1.068966,
 ('face', 'value'): 3.0

In [4]:
import pdb

class Composition(nn.Module):
    def forward(self, x, y):
        return x + y
    
def ranks(arr):
    return np.array(arr).argsort().argsort()

def measure_comp(context, suffix):
    pdb.set_trace()
    print('----')
    print("context: "'%s%s' % (context, suffix))
    vecs = {}
    with open('rep2_data/model%d%s.vec' % (context, suffix)) as f:
        for line in f:
            word, *vec = line.split()
            vec = np.asarray([float(v) for v in vec])
            if SEP in word:
                word = tuple(word.split(SEP))
            if word in bigrams or word in unigrams:
                assert word not in vecs
                vecs[word] = vec
    pdb.set_trace()
    eprim = []
    rprim = []
    ecomp = []
    rcomp = []
    for w1, w2 in bigrams:
        b = (w1, w2)
        if not (w1 in vecs and w2 in vecs and b in vecs):
            continue
        eprim.append(w1)
        rprim.append(vecs[w1])
        eprim.append(w2)
        rprim.append(vecs[w2])
        ecomp.append(b)
        rcomp.append(vecs[b])
    pdb.set_trace()
    errs = evals2.evaluate(rprim + rcomp, eprim + ecomp, Composition(), evals2.CosDist(), zero_init=False)
    pdb.set_trace()
    errs = errs[-len(ecomp):]
    
    scores = [bigrams[e] for e in ecomp]
    
    r_errs = ranks(errs)
    r_scores = ranks(scores)
    
    data = DataFrame({'err': r_errs, 'score': r_scores})
    sns.lmplot(x='err', y='score', data=data)
    plt.xlabel('TRE (rank)')
    plt.ylabel('compositionality (rank)')
    #plt.ylim(0, 5)
    plt.savefig('rep_ctx%d.pdf' % context, format='pdf')
    plt.show()
    print(scipy.stats.spearmanr(errs, scores))
    
    comb = zip(scores, errs, ecomp)
    comb = sorted(comb, key=lambda x: x[1])
    print("compositional:")
    print("%20s %20s %40s" % ("human score", "model err", "words"))
    for c in comb[:5]:
        print("%20.2f %20.2f %40s" % c)
    print()
    print("non-compositional:")
    print("%20s %20s %40s" % ("human score", "model err", "words"))
    for c in comb[-5:]:
        print("%20.2f %20.2f %40s" % c)

In [5]:
for context in [7]: #[1, 3, 5, 7]:
    for suffix in ['']: # ['', '_tiny']:
        measure_comp(context, suffix)

> <ipython-input-4-c50231d04010>(12)measure_comp()
-> print('----')
(Pdb) 
(Pdb) c
----
context: 7
> <ipython-input-4-c50231d04010>(25)measure_comp()
-> eprim = []
(Pdb) c
> <ipython-input-4-c50231d04010>(40)measure_comp()
-> errs = evals2.evaluate(rprim + rcomp, eprim + ecomp, Composition(), evals2.CosDist(), zero_init=False)
(Pdb) c
> /Users/hanozbhathena/Dropbox/scpd/cs224u/code/cs224u/tre/evals2.py(60)evaluate()
     59     pdb.set_trace()
---> 60     for t in range(steps):
     61         opt.zero_grad()

ipdb> c
> /Users/hanozbhathena/Dropbox/scpd/cs224u/code/cs224u/tre/evals2.py(32)compose()
     31             pdb.set_trace()
---> 32             ret_tup = (self.compose(ee) for ee in e)
     33             return self.comp(*ret_tup)

ipdb> b 33
Breakpoint 1 at /Users/hanozbhathena/Dropbox/scpd/cs224u/code/cs224u/tre/evals2.py:33
ipdb> c
> /Users/hanozbhathena/Dropbox/scpd/cs224u/code/cs224u/tre/evals2.py(33)compose()
     32             ret_tup = (self.compose(ee) for ee in e)
1

TypeError: forward() missing 2 required positional arguments: 'x' and 'y'